In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.python.client import device_lib
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow

In [71]:
f = open('HerMajestySpeechesDataset/train.txt', 'r', encoding='utf-8')
# text = f.read()
texts = f.readlines()
f.close()

In [72]:
len(texts)

2361

In [73]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts) 

texts2ids = tokenizer.texts_to_sequences(texts)
print ("Texts as IDs:", texts2ids[:10])
ids2texts = tokenizer.sequences_to_texts(texts2ids)
print ("IDs back to texts:", ids2texts[:10])

Texts as IDs: [[1, 180, 998, 4, 38, 583, 40, 2093, 2094, 20, 1164, 529], [20, 8, 162, 245, 7, 999, 24, 423, 1165, 4, 17, 19], [67, 194, 780], [1, 354, 2, 6, 1346, 721, 52, 584, 39, 212, 1, 296, 2, 530, 207, 1000], [1, 180, 67, 194, 5, 780, 2095, 20, 531, 116, 871, 3, 246], [1, 194, 48, 2096, 5, 2950, 1166, 5, 150, 223, 195, 490, 532, 2097, 196, 20, 6, 2951, 2952], [7, 12, 1167, 204, 872, 4, 113, 9, 454, 186, 297, 585, 1, 258, 10, 110, 204], [57, 2, 1, 454, 9, 28, 49, 648, 19, 9, 78, 10, 65, 11, 1, 311, 2, 67], [16, 270, 6, 58, 82, 7, 781, 4, 455, 2098, 1, 2953, 2, 1, 355, 3, 213, 9, 873, 48, 491, 4, 236, 128, 3, 492, 4, 6, 874, 41], [1, 354, 2, 782, 2099, 2954, 15, 456, 4, 19, 42, 491, 247, 3, 90, 1, 68, 2955, 10, 140]]
IDs back to texts: ["the queen's messages to those celebrating their 90th birthdays on 21 april", 'on this shared occasion i send my warm congratulations to you all', 'christmas broadcast 2006', 'the birth of a baby brings great happiness but then the business of growin

In [66]:
print(len(texts2ids))

2294


In [15]:
test = {x:{} for x in [(1,2),(3,4),(5,6)]}
test

{(1, 2): {}, (3, 4): {}, (5, 6): {}}

In [16]:
test[(1,2)] = {'a':1, 'b':2}
test[(3,4)] = {'a':3, 'b':4}
test

{(1, 2): {'a': 1, 'b': 2}, (3, 4): {'a': 3, 'b': 4}, (5, 6): {}}

In [17]:
ocurrency_example = {
        '': {'a':1, 'b':2},
        'a': {'b': 1, 'c': 1},
        ('a','b'): {'c': 1},
        ('b','c'): {'a': 2, 'd': 1},
        ('c','d'): {'a': 1, 'b': 1, 'c': 1},
}

In [18]:
def toyLM_ngram(ngram_list, ngram_sizes=[2]):
    table_list = []
    for size in ngram_sizes:
        oc_table = {}
        for i in range(len(ngram_list)-size):
            current_ngram = ngram_list[i:i+size]
            for current_ngram in ngram_list:
                if current_ngram in oc_table:
                    oc_table[current_ngram] += 1
                else:
                    oc_table[current_ngram] = 1
        table_list.append(oc_table)
        print(oc_table)
    return table_list

In [41]:
print(toyLM_ngram(texts2ids[0]))


{1: 123, 180: 41, 998: 41, 4: 82, 38: 41, 583: 41, 40: 41, 2093: 41, 2094: 41, 20: 82, 1164: 41, 529: 41, 8: 41, 162: 41, 245: 41, 7: 41, 999: 41, 24: 41, 423: 41, 1165: 41, 17: 41, 19: 41, 67: 41, 194: 41, 780: 41, 354: 41, 2: 82, 6: 41, 1346: 41, 721: 41, 52: 41, 584: 41, 39: 41, 212: 41, 296: 41, 530: 41, 207: 41, 1000: 41}
[{1: 123, 180: 41, 998: 41, 4: 82, 38: 41, 583: 41, 40: 41, 2093: 41, 2094: 41, 20: 82, 1164: 41, 529: 41, 8: 41, 162: 41, 245: 41, 7: 41, 999: 41, 24: 41, 423: 41, 1165: 41, 17: 41, 19: 41, 67: 41, 194: 41, 780: 41, 354: 41, 2: 82, 6: 41, 1346: 41, 721: 41, 52: 41, 584: 41, 39: 41, 212: 41, 296: 41, 530: 41, 207: 41, 1000: 41}]


In [74]:
def get_ngrams(frase, size=2):
    for i in range(size):
        frase = [0] + frase
    ngrams_list = []
    for i in range(len(frase)-size):
        ngrams_list.append((tuple(frase[i:i+size]),frase[i+size]))
    return ngrams_list

In [46]:
print(texts2ids[1])

[1, 180, 67, 194, 5, 780, 2095, 20, 531, 116, 871, 3, 246]


In [75]:
all_ngrams = []
for i in texts2ids:
    all_ngrams += get_ngrams(i)

In [76]:
def co_table(lista_ocurrencias):
    table = {}
    for i in lista_ocurrencias:
        if i[0] in table:
            if i[1] in table[i[0]]:
                table[i[0]][i[1]] += 1
            else:
                table[i[0]][i[1]] = 1
        else:
            table[i[0]] = {}
    return table



In [79]:
sum(co_table(all_ngrams)[(0,0)].values())

2355